---
Title : Classification de l'Audio  
Author : Dmitrašinović Théotime  
Date : 05/11/2023  
**But** :  

1. Création de toutes les combinaisons de :
  - Labelisations
  - Caractéristiques audio
  - Méthodes d'équilibrage du dataset
2. Et application de tous les modèles de Classification:
  - SVC
  - OneVsRest
  - Naive Bayes
3. Puis Affichage des scores et matrice de confusions  

*Des classes python sont définies pour faciliter les combinaisons.*

---

# Classification des caractéristiques Audio

## Google Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Les différentes préparations du Corpus en DataSets

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

In [ ]:
# DataFrame des individus labélisés
CSVLabelisedDataPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/indice_vid_labelled.csv"
df_labels = pd.read_csv(CSVLabelisedDataPath)
CSV_NLP = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP.csv"
df_NLP = pd.read_csv(CSV_NLP, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLP_WoC = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP_WoC.csv"
df_NLP_WoC = pd.read_csv(CSV_NLP_WoC, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLPP = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLPP.csv"
df_NLPP = pd.read_csv(CSV_NLPP, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLPS = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLPS.csv"
df_NLPS = pd.read_csv(CSV_NLPS, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLP_A1k = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP_A1k.csv"
df_NLP_A1k = pd.read_csv(CSV_NLP_A1k, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})

In [ ]:
class Corpus:
  #df_labels = pd.read_csv(CSVLabelisedDataPath)


  def __init__(self, embeddings, equilibrage, l, s, c, df_labels):
    self.df_labels = df_labels
    self.nom = l + "|" + c + " | " + s
    self.nom_equilibrage = s
    self.nom_label = l
    self.nom_cara = c

    self.embeddings = embeddings
    # equilibrage
    self.train, self.test, self.valid = equilibrage(self.df_labels.copy(deep=True))
    # jointure avec les embeddings
    self.join()
    # Séparation de X et y
    self.Xy()


  def join(self):
    self.train = self.train.set_index('name').join(self.embeddings)
    self.test = self.test.set_index('name').join(self.embeddings)
    self.valid = self.valid.set_index('name').join(self.embeddings)

  def Xy(self):
    self.Xtrain = self.train.drop(columns=['label'])
    self.ytrain = self.train.label
    self.Xtest = self.test.drop(columns=['label'])
    self.ytest = self.test.label
    self.Xvalid = self.valid.drop(columns=['label'])
    self.yvalid = self.valid.label

def split_TTV(df):
  # Split the data into train, test, and validation sets while maintaining the class distribution
  # Adjust the test_size and random_state parameters as needed
  train, temp = train_test_split(df, train_size=0.6, stratify=df.label, random_state=42)
  test, val = train_test_split(temp, test_size=0.5, stratify=temp.label, random_state=42)
  return train, test, val

#Séparation 1
def not_balanced(df):
  return split_TTV(df)

#Séparation 2
def delHalfEduc(df_labels):
  # on supprime la moitié des individus avec label Education car sûr représentés

  subset_to_del = df_labels[df_labels['label']=="Education"].sample(len(df_labels[df_labels['label']=="Education"])//2)
  df_labels = df_labels.drop(subset_to_del.index)
  return split_TTV(df_labels)
#Séparation 2
def delHalfAccess(df_labels):
  # on supprime la moitié des individus avec label Education car sûr représentés

  subset_to_del = df_labels[df_labels['label']=="accessibility"].sample(len(df_labels[df_labels['label']=="accessibility"])//2)
  df_labels = df_labels.drop(subset_to_del.index)
  return split_TTV(df_labels)


#Séparation 3
# SurSampling x2 et x3
def surSamplingBasic(df, seuil=4):
  vc = df['label'].value_counts()
  sx2 = int(np.ceil(len(df)*0.02))
  lx2 = vc[vc < sx2].index
  sx3 = int(np.ceil(len(df)*0.05))
  lx3 = vc[vc < sx3].index
  # créer une copie du df auquel on rajoute des nouveaux individus
  balanced_df = df.copy(deep=True)
  for l in lx2:
    label_data = df[df['label'] == l]
    sx2 = min(len(label_data)*2, sx2)
    to_add_subset = label_data.sample(sx2-len(label_data), replace=True)
    balanced_df = pd.concat([balanced_df, to_add_subset])
  for l in lx3:
    label_data = df[df['label'] == l]
    sx3 = min(len(label_data)*3, sx3)
    to_add_subset = label_data.sample(sx3-len(label_data), replace=True)
    balanced_df = pd.concat([balanced_df, to_add_subset])
  return balanced_df
def sursamplingx2x3(df, seuil=0.05):
  train, test, val = split_TTV(df)
  train = surSamplingBasic(train, seuil)
  test = surSamplingBasic(test, seuil)
  valid = surSamplingBasic(val, seuil)
  return train, test, valid

#Séparation 4
# SurSampling et SousSampling
def SurEtSousSampling(df):
  # on supprime la moitié des individus avec label Education car sûr représentés
  train, test, valid = delHalfEduc(df)
  train = surSamplingBasic(train, 0.05)
  test = surSamplingBasic(test, 0.05)
  valid = surSamplingBasic(valid, 0.05)
  return train, test, valid

#equilibrage 4.5
# redistribution pour autos & vehicules
def redistribAuto(df):
  true_Voiture_label = {"-AwRAfxBub9M":"Film & Animation",
                        "-1VM2eLhvsSM":"Film & Animation",
                        "-DfPMxdHZKsw":"People & Blogs",
                        "-XsVV0t_cS7Q":"People & Blogs",
                        "-GHDz-XDD8OU":"Film & Animation",
                        "-PXJnZMF3ucc":"People & Blogs",
                        "-CeN6PMEOQEA":"People & Blogs",
                        "-q6rAllJAdWk":"People & Blogs",
                        "-jHCA59ia_NE":"People & Blogs",
                        "-VsJKPdOuzZw":"Comedy"}
  for id in true_Voiture_label:
    df.loc[df.name==id, ['label']] = true_Voiture_label[id]
  # on delete Autos & Vehicles
  return df.drop(df[df['label']=="Autos & Vehicles"].index)
def redisAutoEtSurSousSample(df):
  df = redistribAuto(df)
  return SurEtSousSampling(df)


#Séparation 5
# changement de label pour les classes minoritaires
def redistributionTOT(df):
  switch_label = {"Gaming":"Entertainment", "Travel & Events":"People & Blogs", "Nonprofits & Activism": "News & Politics"}
  for old_label in switch_label:
    df.loc[df.label==old_label, ['label']] = switch_label[old_label]
  df = redistribAuto(df)
  return df
def redisEtSurSous(df):
  df = redistributionTOT(df)
  return SurEtSousSampling(df)

## Les diférentes caractéristiques Audio et différentes concaténations

In [ ]:
# Les embeddings Audio
# VGGish
AverageEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_average_VGGish.pkl"
MaxPoolEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_max_pooling_VGGish.pkl"
# YAMNet
AveragePredictionsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/YAMNet/ALL_predictions_YAMNet.pkl"

def getEmbesDF(path):
  with open(path, "rb") as fp:
      embeddings = pickle.load(fp)
  df_embes = pd.DataFrame.from_dict((embeddings), orient='index')
  return df_embes

df_avg_embes = getEmbesDF(AverageEmbeddingsPath)
df_max_pool_embes = getEmbesDF(MaxPoolEmbeddingsPath)
df_avg_preds = getEmbesDF(AveragePredictionsPath)

# Différentes Concaténations des 3 vecteurs (avg_embes, max_pool_embes et avg_preds)
# un offset quelconque pour renommer le nom des colonnes
# VGGish max_pool & VGGish avg embes
df_max_pool_et_avg_embes = df_avg_embes.join(df_max_pool_embes.rename(columns={k:1000+k for k in df_max_pool_embes.columns}))
# VGGish max_pool & YAMNet avg predictions
df_max_pool_et_avg_preds = df_avg_preds.join(df_max_pool_embes.rename(columns={k:1000+k for k in df_max_pool_embes.columns}))
# VGGish max_pool & VGGish avg embes & YAMNet avg predictions
df_VGGish_et_YAMNet = df_max_pool_et_avg_embes.join(df_avg_preds.rename(columns={k:2000+k for k in df_avg_preds.columns}))
# VGGish max_pool + VGGish avg embes
df_addition_VGGish = df_avg_embes + df_max_pool_embes
# YAMNet avg predictions & (VGGish max_pool + VGGish avg embes)
df_addition_VGGish_et_YAMNet = df_addition_VGGish.join(df_avg_preds.rename(columns={k:1000+k for k in df_avg_preds.columns}))

In [ ]:
Equilibrages = {"Not balanced":not_balanced, "Halfed education":delHalfEduc, "Sur Sample x2x3":sursamplingx2x3, "Sur & Sous Sample":SurEtSousSampling,"Redistribution Autos & Vehicules": redisAutoEtSurSousSample, "Redistribution Small labels & Sur Sous Sample":redisEtSurSous, "redistribAuto":redistribAuto}
Equilibrages = {"Not balanced":not_balanced, "Sur Sample x2x3":sursamplingx2x3, "Halfed accessibility":delHalfEduc,"Sur & Sous Sample":SurEtSousSampling}

Caracteristiques = {"Average VGGish":df_avg_embes, "Max_pool VGGish":df_max_pool_embes, "Average Environment Sound":df_avg_preds}

CSV_labels = {"Labels A1k":df_NLP_A1k}

# Constructions de toutes la combinaisons de Séparations et Caractéristiques extraites de l'Audio
List_Corpus = []
for l in CSV_labels:
  for s in Equilibrages:
    for c in Caracteristiques:
      print(s, c, l)
      List_Corpus.append(Corpus(embeddings=Caracteristiques[c], equilibrage=Equilibrages[s], l=l, s=s, c=c, df_labels=CSV_labels[l]))


Not balanced Average VGGish Labels A1k
Not balanced Max_pool VGGish Labels A1k
Not balanced Average Environment Sound Labels A1k
Sur Sample x2x3 Average VGGish Labels A1k
Sur Sample x2x3 Max_pool VGGish Labels A1k
Sur Sample x2x3 Average Environment Sound Labels A1k
Halfed accessibility Average VGGish Labels A1k
Halfed accessibility Max_pool VGGish Labels A1k
Halfed accessibility Average Environment Sound Labels A1k
Sur & Sous Sample Average VGGish Labels A1k
Sur & Sous Sample Max_pool VGGish Labels A1k
Sur & Sous Sample Average Environment Sound Labels A1k


In [ ]:
sep = "Redistribution Small labels & Sur Sous Sample"
car = "Max_pool VGGish"
corpus = Corpus(Caracteristiques[car], Equilibrages[sep], sep + car)
px.pie(corpus.train, names="label", title=corpus.nom)

## Les différents modèles de Classification

In [ ]:
class Classif:

  def __init__(self, corpus, nom_model, accuracy, predictions):
    self.corpus = corpus
    self.nom = self.corpus.nom + '|' + nom_model
    self.nom_model = nom_model
    self.accuracy = accuracy
    self.predictions = predictions
    self.matrice_confusion = conf_matrix(self.corpus.ytest, self.predictions)

In [ ]:
from sklearn.metrics import confusion_matrix

def conf_matrix(y_test, pred):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  return cf

def show_conf_matrix(y_test, pred):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  data = cf
  fig = px.imshow(data,
                  labels=dict(y="Vrai Catégorie", x="Catégorie prédite", color="Productivity"),
                  x=list_cat,
                  y=list_cat
                )
  fig.update_xaxes(side="top")
  fig.show()

## Tous les corpus et tous les modèles

In [ ]:
# pipeline & normalisation
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# SVC & OneVsRest & NaiveBayes
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB

Models = {'SVC':make_pipeline(StandardScaler(), SVC(gamma='auto')),
          '1vsRestSVC':make_pipeline(StandardScaler(), OneVsRestClassifier(SVC(gamma='auto'))),
          'Naive Bayes':make_pipeline(StandardScaler(),GaussianNB())
          }


In [ ]:
# tous les corpus et tous les modèles
List_Classif = []
for corpus in List_Corpus:
  for nom_model in Models:
    model = Models[nom_model]
    model.fit(corpus.Xtrain, corpus.ytrain)
    predictions = model.predict(corpus.Xtest)
    accuracy = model.score(corpus.Xtest, corpus.ytest)
    List_Classif.append(Classif(corpus, nom_model, accuracy, predictions))

In [ ]:
len(List_Classif)

36

### Matrice confusion pour ``nom_equilibrage = "Halfed accessibility" OU "Sur & Sous Sample"``

In [ ]:
for i, classif in enumerate(List_Classif):
  if classif.corpus.nom_equilibrage in ["Halfed accessibility","Sur & Sous Sample"]:
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)

Labels A1k|Average VGGish | Halfed accessibility|SVC
accuracy: 0.31843575418994413


Labels A1k|Average VGGish | Halfed accessibility|1vsRestSVC
accuracy: 0.2849162011173184


Labels A1k|Average VGGish | Halfed accessibility|SGDC
accuracy: 0.22625698324022347


Labels A1k|Max_pool VGGish | Halfed accessibility|SVC
accuracy: 0.3575418994413408


Labels A1k|Max_pool VGGish | Halfed accessibility|1vsRestSVC
accuracy: 0.2988826815642458


...

Labels A1k|Max_pool VGGish | Sur & Sous Sample|1vsRestSVC
accuracy: 0.2967914438502674


Labels A1k|Max_pool VGGish | Sur & Sous Sample|SGDC
accuracy: 0.1711229946524064


Labels A1k|Average Environment Sound | Sur & Sous Sample|SVC
accuracy: 0.29411764705882354


Labels A1k|Average Environment Sound | Sur & Sous Sample|1vsRestSVC
accuracy: 0.2620320855614973


Labels A1k|Average Environment Sound | Sur & Sous Sample|SGDC
accuracy: 0.22192513368983957


### Matrice confusion pour ``classif.accuracy > 0.40``

In [ ]:
for i, classif in enumerate(List_Classif):
  if classif.accuracy > 0.40:
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)

Labels A1k|Average VGGish | Not balanced|SVC
accuracy: 0.4451754385964912


Labels A1k|Average VGGish | Not balanced|1vsRestSVC
accuracy: 0.42543859649122806


Labels A1k|Max_pool VGGish | Not balanced|SVC
accuracy: 0.4451754385964912


Labels A1k|Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.4342105263157895


Labels A1k|Average Environment Sound | Not balanced|SVC
accuracy: 0.43640350877192985


Labels A1k|Average Environment Sound | Not balanced|1vsRestSVC
accuracy: 0.41228070175438597


Labels A1k|Average VGGish | Sur Sample x2x3|SVC
accuracy: 0.4077079107505071


Labels A1k|Max_pool VGGish | Sur Sample x2x3|SVC
accuracy: 0.40973630831643004


Labels A1k|Average Environment Sound | Sur Sample x2x3|SVC
accuracy: 0.40162271805273836


In [ ]:
Equilibrages = {"Not balanced":not_balanced,
               "Halfed education":delHalfEduc,
               "Sur Sample x2x3":sursamplingx2x3,
               "Sur & Sous Sample":SurEtSousSampling}
               #"Redistribution Small labels & Sur Sous Sample":redisEtSurSous}

Caracteristiques = {"VGGish & YAMNet":df_VGGish_et_YAMNet}

CSV_labels = {"Labels Normaux":df_labels, "Labels NLP":df_NLP, "Labels NLP_WoC":df_NLP_WoC}
CSV_labels = {"Labels Normaux":df_labels, "NLP Proba":df_NLPP, "NLP scricte":df_NLPS}

# Constructions de toutes la combinaisons de Séparations et Caractéristiques extraites de l'Audio
List_Corpus = []
for l in CSV_labels:
  for s in Equilibrages:
    for c in Caracteristiques:
      print(s, c, l)
      List_Corpus.append(Corpus(embeddings=Caracteristiques[c], equilibrage=Equilibrages[s], l=l, s=s, c=c, df_labels=CSV_labels[l]))
print("\n-", len(CSV_labels)*len(Equilibrages)*len(Caracteristiques), "Combinaisons possibles pour lesquelles on va essayer chaque modèle de Classification")

Not balanced VGGish & YAMNet Labels Normaux
Halfed education VGGish & YAMNet Labels Normaux
Sur Sample x2x3 VGGish & YAMNet Labels Normaux
Sur & Sous Sample VGGish & YAMNet Labels Normaux
Not balanced VGGish & YAMNet NLP Proba
Halfed education VGGish & YAMNet NLP Proba
Sur Sample x2x3 VGGish & YAMNet NLP Proba
Sur & Sous Sample VGGish & YAMNet NLP Proba
Not balanced VGGish & YAMNet NLP scricte
Halfed education VGGish & YAMNet NLP scricte
Sur Sample x2x3 VGGish & YAMNet NLP scricte
Sur & Sous Sample VGGish & YAMNet NLP scricte

- 12 Combinaisons possibles pour lesquelles on va essayer chaque modèle de Classification


## Résultats

In [ ]:
# METHODES D'EQUILIBRAGE DU DATASET
Equilibrages = {"Not balanced":not_balanced,
               "Halfed education":delHalfEduc,
               "Sur Sample x2x3":sursamplingx2x3,
               "Sur & Sous Sample":SurEtSousSampling,
               "Redistribution Autos & Vehicules": redisAutoEtSurSousSample,
               "Redistribution Small labels & Sur Sous Sample":redisEtSurSous
               }

# CARACTERISTIQUES AUDIO
Caracteristiques = {"Average VGGish":df_avg_embes,
                    "Max_pool VGGish":df_max_pool_embes,
                    "Average Environment Sound":df_avg_preds,
                    "sum(VGGish)":df_addition_VGGish,
                    "Average VGGish et Max_pool VGGish":df_max_pool_et_avg_embes,
                    "YAMNet et Max_pool VGGish":df_max_pool_et_avg_preds,
                    "VGGish & YAMNet":df_VGGish_et_YAMNet,
                    "sum(VGGish) & YAMNet":df_addition_VGGish_et_YAMNet}

# LABELS
CSV_labels = {"Labels Normaux":df_labels, "Labels NLP":df_NLP, "Labels NLP_WoC":df_NLP_WoC}
CSV_labels = {"Labels Normaux":df_labels, "NLP Proba":df_NLPP, "NLP scricte":df_NLPS}
CSV_labels = {"Labels A1k":df_NLP_A1k}
CSV_labels = {"Labels Normaux":df_labels}

# Constructions de toutes les combinaisons de Séparations et Caractéristiques extraites de l'Audio avec potentielement labels différents
List_Corpus = []
for l in CSV_labels:
  for s in Equilibrages:
    for c in Caracteristiques:
      print(s, c, l)
      List_Corpus.append(Corpus(embeddings=Caracteristiques[c], equilibrage=Equilibrages[s], l=l, s=s, c=c, df_labels=CSV_labels[l]))
print("\n-", len(CSV_labels)*len(Equilibrages)*len(Caracteristiques), "Combinaisons possibles pour lesquelles on va essayer chaque modèle de Classification")

Not balanced Average VGGish Labels Normaux
Not balanced Max_pool VGGish Labels Normaux
Not balanced Average Environment Sound Labels Normaux
Not balanced sum(VGGish) Labels Normaux
Not balanced Average VGGish et Max_pool VGGish Labels Normaux
Not balanced YAMNet et Max_pool VGGish Labels Normaux
Not balanced VGGish & YAMNet Labels Normaux
Not balanced sum(VGGish) & YAMNet Labels Normaux
Halfed education Average VGGish Labels Normaux
Halfed education Max_pool VGGish Labels Normaux
Halfed education Average Environment Sound Labels Normaux
Halfed education sum(VGGish) Labels Normaux
Halfed education Average VGGish et Max_pool VGGish Labels Normaux
Halfed education YAMNet et Max_pool VGGish Labels Normaux
Halfed education VGGish & YAMNet Labels Normaux
Halfed education sum(VGGish) & YAMNet Labels Normaux
Sur Sample x2x3 Average VGGish Labels Normaux
Sur Sample x2x3 Max_pool VGGish Labels Normaux
Sur Sample x2x3 Average Environment Sound Labels Normaux
Sur Sample x2x3 sum(VGGish) Labels Nor

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import GaussianNB

from tqdm.notebook import tqdm_notebook

# tous les corpus et tous les modèles
List_Classif = []
for corpus in tqdm_notebook(List_Corpus, desc='Corpus'):
  for nom_model in tqdm_notebook(Models, desc='Models', leave=False):
    model = Models[nom_model]
    model.fit(corpus.Xtrain, corpus.ytrain)
    predictions = model.predict(corpus.Xtest)
    accuracy = model.score(corpus.Xtest, corpus.ytest)
    List_Classif.append(Classif(corpus, nom_model, accuracy, predictions))

Corpus:   0%|          | 0/48 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

...

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
acc = [c.accuracy for c in List_Classif]
i_sorted = np.argsort(acc)
for i in reversed(i_sorted[-10:]):
  print(List_Classif[i].nom)
  print("accuracy:",List_Classif[i].accuracy)
  show_conf_matrix(List_Classif[i].corpus.ytest, List_Classif[i].predictions)

Labels Normaux|Average VGGish et Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.5241228070175439


Labels Normaux|Average VGGish | Not balanced|1vsRestSVC
accuracy: 0.5131578947368421


Labels Normaux|Average VGGish | Not balanced|SVC
accuracy: 0.5109649122807017


Labels Normaux|sum(VGGish) | Not balanced|1vsRestSVC
accuracy: 0.5087719298245614


Labels Normaux|Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.506578947368421


Labels Normaux|Average VGGish et Max_pool VGGish | Not balanced|SVC
accuracy: 0.506578947368421


Labels Normaux|sum(VGGish) | Not balanced|SVC
accuracy: 0.49780701754385964


Labels Normaux|Max_pool VGGish | Not balanced|SVC
accuracy: 0.4934210526315789


Labels Normaux|VGGish & YAMNet | Not balanced|1vsRestSVC
accuracy: 0.49122807017543857


Labels Normaux|Average VGGish et Max_pool VGGish | Halfed education|1vsRestSVC
accuracy: 0.4725848563968668


In [ ]:
print(len(List_Classif))
for i, classif in enumerate(List_Classif):
  if classif.accuracy > 0.50:
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)

144
Labels Normaux|Average VGGish | Not balanced|SVC
accuracy: 0.5109649122807017


Labels Normaux|Average VGGish | Not balanced|1vsRestSVC
accuracy: 0.5131578947368421


Labels Normaux|Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.506578947368421


Labels Normaux|sum(VGGish) | Not balanced|1vsRestSVC
accuracy: 0.5087719298245614


Labels Normaux|Average VGGish et Max_pool VGGish | Not balanced|SVC
accuracy: 0.506578947368421


Labels Normaux|Average VGGish et Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.5241228070175439


In [ ]:
print(len(List_Classif))
for i, classif in enumerate(List_Classif):
  if classif.accuracy > 0.45 and classif.accuracy < 0.5:
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)

144
Labels Normaux|Max_pool VGGish | Not balanced|SVC
accuracy: 0.4934210526315789


Labels Normaux|sum(VGGish) | Not balanced|SVC
accuracy: 0.49780701754385964


Labels Normaux|YAMNet et Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.47149122807017546


Labels Normaux|VGGish & YAMNet | Not balanced|SVC
accuracy: 0.45394736842105265


Labels Normaux|VGGish & YAMNet | Not balanced|1vsRestSVC
accuracy: 0.49122807017543857


Labels Normaux|sum(VGGish) & YAMNet | Not balanced|1vsRestSVC
accuracy: 0.47149122807017546


Labels Normaux|Average VGGish | Halfed education|1vsRestSVC
accuracy: 0.45691906005221933


Labels Normaux|Average VGGish et Max_pool VGGish | Halfed education|1vsRestSVC
accuracy: 0.4725848563968668


Labels Normaux|Max_pool VGGish | Sur Sample x2x3|1vsRestSVC
accuracy: 0.4638447971781305


Labels Normaux|sum(VGGish) | Sur Sample x2x3|1vsRestSVC
accuracy: 0.4567901234567901


In [ ]:
print(len(List_Classif))
for i, classif in enumerate(List_Classif):
  if classif.nom[:3] == 'NLP':
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)



120
NLP Proba|Average VGGish | Not balanced|SVC
accuracy: 0.28728070175438597


NLP Proba|Average VGGish | Not balanced|1vsRestSVC
accuracy: 0.25877192982456143


NLP Proba|Max_pool VGGish | Not balanced|SVC
accuracy: 0.2807017543859649


NLP Proba|Max_pool VGGish | Not balanced|1vsRestSVC
accuracy: 0.24342105263157895


NLP Proba|Average Environment Sound | Not balanced|SVC
accuracy: 0.2412280701754386


...

NLP scricte|Average Environment Sound | Sur & Sous Sample|1vsRestSVC
accuracy: 0.21729957805907174


NLP scricte|Average VGGish + Max_pool VGGish | Sur & Sous Sample|SVC
accuracy: 0.2742616033755274


NLP scricte|Average VGGish + Max_pool VGGish | Sur & Sous Sample|1vsRestSVC
accuracy: 0.24472573839662448


NLP scricte|YAMNet et Max_pool VGGish | Sur & Sous Sample|SVC
accuracy: 0.24050632911392406


NLP scricte|YAMNet et Max_pool VGGish | Sur & Sous Sample|1vsRestSVC
accuracy: 0.22784810126582278


In [ ]:
List_Classif[]